In [8]:
import os
os.environ['LD_LIBRARY_PATH'] = ':/root/.mujoco/mujoco200/bin'
# os.environ['LD_PRELOAD'] = ':'
import numpy as np
import robosuite as suite
from robosuite.utils.mjcf_utils import postprocess_model_xml
import tensorflow as tf

2023-05-03 02:47:41.829863: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-03 02:47:41.832019: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-03 02:47:41.879683: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-03 02:47:41.881072: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-03 02:47:42.721861: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [9]:
import os
import h5py
import argparse
import random

In [10]:
demo_path = '/home/ar4451/Data/RoboTurkPilot/bins-full/'
hdf5_path = "/home/ar4451/Data/RoboTurkPilot/bins-full/demo.hdf5"
f = h5py.File(hdf5_path, "r")
env_name = f["data"].attrs["env"]
demos = list(f["data"].keys())

In [11]:
env = suite.make(
        env_name,
        has_renderer=False,
        ignore_done=True,
        use_camera_obs=False,
        gripper_visualization=True,
        reward_shaping=True,
        control_freq=100,
    )

In [38]:
# Model Definition
tf_model = tf.keras.Sequential()

tf_model.add(tf.keras.layers.Dense(512, input_shape=(73,)))
tf_model.add(tf.keras.layers.Activation(tf.nn.relu))
tf_model.add(tf.keras.layers.Dropout(0.5))
tf_model.add(tf.keras.layers.Dense(256))
tf_model.add(tf.keras.layers.Activation(tf.nn.elu))
tf_model.add(tf.keras.layers.Dropout(0.5))
tf_model.add(tf.keras.layers.Dense(512))
tf_model.add(tf.keras.layers.Activation(tf.nn.elu))
tf_model.add(tf.keras.layers.Dense(8,activation="softmax",
    kernel_regularizer=tf.keras.regularizers.l2(l=0.01)))

tf_model.summary()
tf_model.compile(loss='mean_absolute_error',
     optimizer='adam',
     metrics=['accuracy'])

indirect_data = True

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 512)               37888     
                                                                 
 activation_18 (Activation)  (None, 512)               0         
                                                                 
 dropout_12 (Dropout)        (None, 512)               0         
                                                                 
 dense_25 (Dense)            (None, 256)               131328    
                                                                 
 activation_19 (Activation)  (None, 256)               0         
                                                                 
 dropout_13 (Dropout)        (None, 256)               0         
                                                                 
 dense_26 (Dense)            (None, 512)              

In [40]:
rewards = []
rewards2 = []
for i in range(10):
    ep = random.choice(demos)

    # read the model xml, using the metadata stored in the attribute for this episode
    model_file = f["data/{}".format(ep)].attrs["model_file"]
    model_path = os.path.join(demo_path, "models", model_file)
    with open(model_path, "r") as model_f:
        model_xml = model_f.read()

    states = f["data/{}/states".format(ep)][()]

    jvels = f["data/{}/joint_velocities".format(ep)][()]
    grip_acts = f["data/{}/gripper_actuations".format(ep)][()]
    actions = np.concatenate([jvels, grip_acts], axis=1)
    num_actions = actions.shape[0]
    ep_rewards = []
    tf_model.fit(states, actions, batch_size=256, validation_split=0.1, epochs=30)
    
    '''
    Evaluation
    if indirect_data:
        for j, action in enumerate(actions):
            state = env.sim.get_state().flatten()
            state = np.array([state])
            predicted = tf_model.predict(state)
            predicted = np.squeeze(predicted)
            obs, reward, done, _  = env.step(predicted)
            ep_rewards.append(reward)
            if done:
                break
    else:
        for state, action in zip(states, actions):
            env.sim.set_state_from_flattened(state)
            env.sim.forward()
            ep_rewards.append(env.reward(action))
    
    rewards.append(ep_rewards)
    '''


Epoch 1/30
17/17 [==============================] - 0s 16ms/step - loss: 0.2521 - accuracy: 0.5434 - val_loss: 0.2187 - val_accuracy: 0.5673
Epoch 2/30
17/17 [==============================] - 0s 14ms/step - loss: 0.2499 - accuracy: 0.5817 - val_loss: 0.2121 - val_accuracy: 0.6087
Epoch 3/30
17/17 [==============================] - 0s 13ms/step - loss: 0.2485 - accuracy: 0.6054 - val_loss: 0.2117 - val_accuracy: 0.6128
Epoch 4/30
17/17 [==============================] - 0s 12ms/step - loss: 0.2480 - accuracy: 0.6296 - val_loss: 0.2115 - val_accuracy: 0.6190
Epoch 5/30
17/17 [==============================] - 0s 14ms/step - loss: 0.2473 - accuracy: 0.6406 - val_loss: 0.2109 - val_accuracy: 0.6253
Epoch 6/30
17/17 [==============================] - 0s 12ms/step - loss: 0.2470 - accuracy: 0.6432 - val_loss: 0.2107 - val_accuracy: 0.6749
Epoch 7/30
17/17 [==============================] - 0s 11ms/step - loss: 0.2468 - accuracy: 0.6512 - val_loss: 0.2119 - val_accuracy: 0.6584
Epoch 8/30
17

In [41]:
tf_model.save('/home/ar4451/tf/il')

2023-05-03 03:19:23.462507: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,512]
	 [[{{node inputs}}]]
2023-05-03 03:19:23.481662: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,256]
	 [[{{node inputs}}]]
2023-05-03 03:19:23.708530: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,512]
	 [[{{node inputs}}]]
2023-05-03 03:19

INFO:tensorflow:Assets written to: /home/ar4451/tf/il/assets


INFO:tensorflow:Assets written to: /home/ar4451/tf/il/assets
